In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble  import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import random

from sklearn import metrics
from collections import Counter
import argparse
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import ParameterGrid
import lightgbm as lgb
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

In [8]:
df = pd.read_csv('../SC-Dapp-scraping/data/dapps_source_code_parsed_open_source_2021-01-08.csv')

In [9]:
df.license.unique()

array(['GPL-3.0', 'Apache 2.0', 'Apache', 'MIT', 'GPL v3.0', 'GPL',
       'Apache-2.0', 'Apache v2', 'Open',
       'Creative Commons Attribution-NonCommercial-NoDeriv', 'NPOSL-3.0',
       'AGPL V3', 'GPL-3', 'Open source', 'e.g MiningExpert',
       'MIT, GPL-3.0', 'Apache2.0', 'GNU General Public License',
       'GPL3.0', 'WTFPL', 'open source', 'AGPL 3.0', 'No License', 'LTD',
       'Apache License 2.0',
       'Random values licensed for virtual item gambling', 'Mit',
       'EPL-1.0', 'Propietary', 'AGPL-v3', 'GPL 3.0', 'no licences',
       'AGPL', 'open-source', 'Copyright', 'Open Source', 'open',
       'Software license GPL3', 'MIT, GPL', 'IOS', 'GPL3', 'LGPL 3.0',
       'none', 'All rights reserved', 'BSD-3', 'GNU', 'Solidity',
       'GNU GPLv3', nan, 'Commercial', 'No license',
       'Open-Source Licenses', 'GitHub', 'GPL-3.0 License', 'Apache2',
       'Proprietary', 'AMP License', 'Unlicensed', 'Unknown license',
       'Apache 2', 'Wbank develop and technology pte 

In [10]:
df.shape

(1625, 13)